In [1]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/eval_help.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/paper_settings.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/validation_help.R"))

library(tidyverse)
library(survival)
library(pROC)
library(survcomp)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Type 'citation("pROC")' for a citation.

Attaching 

### 0 - Read in external studies and bind rows

In [2]:
validation_measure <- (
    readRDS(paste0(TMP_DIR, "validation-measure.Rds"))
        %>% select(
          patient_id, 
          build, 
          Study_cohort, 
          model, 
          tissue, 
          complete, 
          bor, 
          os_event, 
          os_days,
          os, 
          pred_lr, 
          pred_os        
        )
        %>% filter(!is.na(bor))
)

### 1 - Apply models

In [3]:
measure <- function (df, i, j, k = "all", l = TRUE, q = "1") {
    if (k != "all") {
        tmp <- df %>% filter(Study_cohort == i, model == j, tissue == k, complete == l)
    } else {
        tmp <- df %>% filter(Study_cohort == i, model == j, complete == l)
    }
    
    if (length(unique(tmp$bor)) > 1) {
        #tmp <- tmp %>% filter( !is.na(bor))
        measure_auc <- roc(tmp$bor, tmp$pred_lr, plot = FALSE, auc = TRUE)$auc
    } else {
        measure_auc <- NA
    }
    
    tmp <- tmp %>% drop_na(os_event, os_days)
    
    measure_c_index <- concordance.index(tmp$pred_os, tmp$os_days, tmp$os_event)$c.index
    response_low <- mean(tmp %>% filter(pred_lr < .1, !is.na(bor)) %>% pull(bor))
    n_response_low <- length(tmp %>% filter(pred_lr < .1, !is.na(bor)) %>% pull(bor))
    
    data.frame(
            Study = i, 
            model = j, 
            tissue = k, 
            n = nrow(tmp), 
            complete = l, 
            build = q, 
            auc = measure_auc, 
            c_index = measure_c_index, 
            response_low = response_low, 
            n_response_low = n_response_low
    )
}

### 2 - Measure all cohorts

In [4]:
lets_see <- data.frame()
for( i in unique(validation_measure $Study_cohort)){
    print(i)
    for ( j in unique(validation_measure$model)){
        for( q in unique(validation_measure$build)){
            for ( l in c(TRUE, FALSE)){
                lets_see <- rbind(lets_see, measure( validation_measure, i, j, k="all", l, q))
            }
        }
    }
}

[1] "VHIO"


Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1


[1] "INSPIRE"


Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1


[1] "HMF-CPCT-lung"


...

Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1


[1] "Lyon"


Setting levels: control = 0, case = 1
Setting direction: controls > cases
Setting levels: control = 0, case = 1
Setting direction: controls > cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls > cases
Setting levels: control = 0, case = 1
Setting direction: controls > cases


[1] "HMF-CPCT"


Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1


In [5]:
validation_summary <- (
    lets_see 
        %>% group_by( Study, model, tissue, complete) 
        %>% summarise(auc = mean(auc, na.rm = TRUE), 
                      c_index = mean(c_index, na.rm = TRUE), 
                      non_response = mean(response_low, na.rm = TRUE))
        %>% ungroup()
        %>% rename( features = model)
        %>% gather( model, mn, -Study, -features, -tissue, -complete )
)

`summarise()` has grouped output by 'Study', 'model', 'tissue'. You can override using the `.groups` argument.


### 3 - Clean and pretty

In [6]:
name_map <- list(
    "tmb_bin" = "TMB Binary",
    "base_bin" = "TMB Binary + PDL1",
    "tmb" = "TMB", 
    "base" = "TMB + PDL1",
    "rna" = "RNA only",
    "no_tmb" = "Four factors (No TMB)",
    "five_latent" = "Five Factors",
    "fiver_latent_purity" = "Five Factor (purity adjusted)"
)
validation_summary$clean_name <- factor(unlist(lapply(validation_summary$features, function(i) name_map[[i]])), levels = unname(unlist(name_map))) 

In [7]:
validation_summary$model <- ifelse( validation_summary$model == "auc", "Best Overall Response (AUC)", validation_summary$model)
validation_summary$model <- ifelse( validation_summary$model == "c_index", "Overall Survival (C-Index)", validation_summary$model)
validation_summary$model <- ifelse( validation_summary$model == "non_response", "Predicted Non-responder", validation_summary$model)
validation_summary$five <- ifelse( validation_summary$features %in% c("five_latent", "no_tmb"), "yes", "no")

In [8]:
validation_summary$model <- factor(validation_summary$model, levels = c("Best Overall Response (AUC)","Overall Survival (C-Index)","Predicted Non-responder"))

### 3 - add prediction groups

In [9]:
saveRDS(validation_summary, paste0(I_DIR, "validation-summary.Rds"))